## Домашняя работа к лекции "Основы веб-скрапинга и работы с API"

### Задание 1
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: - -

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from datetime import datetime

In [6]:
KEYWORDS = ['Frontend', 'программист']
def get_articles(words, URL_1 = 'https://habr.com/ru/all/'):
    res = requests.get(URL_1)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('article', class_ = 'post post_preview')
    
    preview = pd.DataFrame()
    
    for news_block in news_blocks:
        for key in KEYWORDS:
            if key.lower() in news_block.text.lower():
                date = news_block.find('span', class_ = 'post__time').text
                title = news_block.find('h2', class_ = 'post__title').text
                link = news_block.find('a', class_ = 'post__title_link').get('href')
                all_info = {'date': date, 'title': title.strip(), 'link': link}
                preview = pd.concat([preview, pd.DataFrame([all_info])])
          
    preview = preview.drop_duplicates()
    return preview
get_articles(KEYWORDS)

,date,title,link
0,сегодня в 14:08,"Библиотека Frontend-разработчика, часть 3: Лит...",https://habr.com/ru/post/536544/
0,сегодня в 13:21,Создаём свою «идеальную программерскую» раскла...,https://habr.com/ru/post/536540/
0,сегодня в 10:04,STM32 Assembler Editor: Гора так и не сдвинула...,https://habr.com/ru/post/536524/


### Задание 2.
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [9]:
EMAIL = ['anna.x.ivanova@gmail']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
addresses = {'emailAddresses': EMAIL}

res = requests.post(url, json=addresses, headers=headers)

df = pd.DataFrame(res.json()['breaches']).T.reset_index() 

df = df[['site', 'description', 'publishDate']]
df['email'] = [', '.join(el.keys()) for el in list(res.json()['data'].values())]

df

TypeError: 'NoneType' object is not subscriptable